# Multi-Qubit Gate Stuff

In [7]:
import numpy as np
from scipy.linalg import expm, sqrtm, svdvals
from scipy.optimize import minimize

import cirq

import sys
sys.path.append("../Gates_Lab_Suite")

from Auto_Algorithm import *
from Core_Definition import *
from Visualization import *

### Native Gates

In [2]:
def Ra(theta, phi):

    theta = theta * np.pi
    phi = phi * np.pi

    return np.array([[np.cos(theta / 2), -1j * np.exp(-1j * phi) * np.sin(theta / 2)], 
                     [-1j * np.exp(1j * phi) * np.sin(theta / 2), np.cos(theta / 2)]])

def Rz(theta):

    theta = theta * np.pi 

    return np.array([[np.exp(-1j * theta / 2), 0],
                     [0, np.exp(1j * theta / 2)]])


def Ry(theta):
    return Ra(theta, 0.5)


def Rx(theta):
    return Ra(theta, 0)


Id = np.array([[1, 0], [0, 1]])


def XX(xi):

    xi = xi * np.pi
    
    return np.array([[np.cos(xi), 0, 0, -1j*np.sin(xi)], 
                     [0, np.cos(xi), -1j*np.sin(xi), 0], 
                     [0, -1j*np.sin(xi), np.cos(xi), 0], 
                     [-1j*np.sin(xi), 0, 0, np.cos(xi)]])

# Common Non-Native Gates

In [4]:
def CNOT(): 
    
    return np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 0, 1],
                     [0, 0, 1, 0]])


def gates_circuit_CNOT():

    m = Quantum_Gate("CNOT", 0, 1).Matrix_Representation(2)
    return m


def CZ():

    return np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, -1]])


def gates_circuit_CZ():

    m = Quantum_Gate("CZ", 0, 1).Matrix_Representation(2)

    return m


def figgatt_circuit_CZ():

    circuit = np.kron(Ry(-0.5), Ry(0.5))
    circuit = np.matmul(np.kron(Rx(-0.5), Rx(0.5)), circuit)
    circuit = np.matmul(XX(0.25), circuit)
    circuit = np.matmul(np.kron(Ry(0.5), Ry(-0.5)), circuit)

    return circuit


def CRy(theta):

    theta *= np.pi

    return np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, np.cos(theta/2), -np.sin(theta/2)],
                     [0, 0, np.sin(theta/2), np.cos(theta/2)]])

# Distance Measures

In [20]:
def check_unitary_equality(U, V):

    return cirq.equal_up_to_global_phase(U, V)


def fidelity(U, V):
  """
  Calculates the fidelity between two density matrices.

  Args:
    rho: First density matrix (NumPy array).
    sigma: Second density matrix (NumPy array).

  Returns:
    The fidelity value (float).
  """
  
  # Calculate the square roots of the density matrices
  rho_sqrt = sqrtm(U)
  sigma_sqrt = sqrtm(V)

  # Calculate the product of the square roots
  product = rho_sqrt @ sigma_sqrt

  # Calculate the singular values of the product
  singular_values = svdvals(product)

  # Calculate the trace norm (sum of singular values)
  trace_norm = singular_values.sum()

  # Square the trace norm to get the fidelity
  fidelity = trace_norm**2

  return fidelity


In [15]:
cirq.equal_up_to_global_phase(figgatt_circuit_CZ(), CZ())

True

In [23]:
fidelity(np.eye(4), CNOT())

15.999999999999996

In [ ]:
figgatt_circuit_CZ() @ CZ().conj().T 

/tmp/ipykernel_41725/2752200329.py:1: RuntimeWarning: divide by zero encountered in divide
  figgatt_circuit_CZ() @ CZ().conj().T / np.eye(4)


array([[0.70710678-0.70710678j,       -inf       -infj,
               inf       -infj,        inf       -infj],
       [      -inf       -infj, 0.70710678-0.70710678j,
              -inf       +infj,        inf       -infj],
       [       inf       -infj,       -inf       +infj,
        0.70710678-0.70710678j,       -inf       +infj],
       [       inf       -infj,       -inf       +infj,
               inf       -infj, 0.70710678-0.70710678j]])

For a given two-qubit gate, find a set of single-qubit and two-qubit gates that implement the same unitary:

In [147]:
def circuit_ansatz_euler_rotations(alpha1, beta1, gamma1, alpha2, beta2, gamma2, alphap1, betap1, gammap1, alphap2, betap2, gammap2, xi_xx):
    
    # Arbitrary single-qubit rotation on both qubits (each qubit can have different angle)
    circuit = np.kron(Rz(alpha1), Rz(alpha2))
    circuit = np.matmul(np.kron(Ry(beta1), Ry(beta2)), circuit)
    circuit = np.matmul(np.kron(Rz(gamma1), Rz(gamma2)), circuit)

    # XX gate of any angle
    circuit = np.matmul(XX(xi_xx), circuit)

    # Arbitrary single-qubit rotation on both qubits (each qubit can have different angle)
    circuit = np.matmul(np.kron(Rz(alphap1), Rz(alphap2)), circuit)
    circuit = np.matmul(np.kron(Ry(betap1), Ry(betap2)), circuit)
    circuit = np.matmul(np.kron(Rz(gammap1), Rz(gammap2)), circuit)

    return circuit / circuit[0][0]

In [148]:
bounds = ((-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (0, 2))
exact_solution = [0, 0.5, 0, 0, 0, 0, 0, -0.5, -0.5, 0.5, 0.5, -0.5, 0.25]


initial_guess = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.25]
initial_guess = exact_solution

In [149]:
def CNOT_optimization_function(x):

    return np.linalg.norm(circuit_ansatz_euler_rotations(*x) - CNOT())



def CZ_optimization_function(x):

    return np.linalg.norm(circuit_ansatz_euler_rotations(*x) - CZ())

In [150]:
print(np.linalg.norm(CZ()-circuit_ansatz_euler_rotations(*exact_solution)))

1.2183981524678704e+16


In [141]:
circuit_ansatz_euler_rotations(*exact_solution)

array([[1.64149953e-16, 1.00000000e+00, 8.42634303e-17, 7.85046229e-17],
       [1.00000000e+00, 1.64149953e-16, 7.85046229e-17, 8.42634303e-17],
       [7.85046229e-17, 8.42634303e-17, 1.00000000e+00, 4.78884530e-17],
       [8.42634303e-17, 7.85046229e-17, 4.78884530e-17, 1.00000000e+00]])

In [142]:
CNOT_optimization_function(exact_solution)

2.8284271247461903

In [145]:
result = minimize(CNOT_optimization_function, initial_guess, bounds=bounds, method="Nelder-Mead")

In [146]:
result

       message: Maximum number of function evaluations has been exceeded.
       success: False
        status: 1
           fun: 0.3817631793587355
             x: [ 9.604e-01  5.030e-01 ...  1.000e+00  2.476e-01]
           nit: 1975
          nfev: 2600
 final_simplex: (array([[ 9.604e-01,  5.030e-01, ...,  1.000e+00,
                         2.476e-01],
                       [ 9.633e-01,  5.052e-01, ...,  1.000e+00,
                         2.477e-01],
                       ...,
                       [ 9.632e-01,  5.035e-01, ...,  1.000e+00,
                         2.478e-01],
                       [ 9.617e-01,  5.031e-01, ...,  1.000e+00,
                         2.478e-01]]), array([ 3.818e-01,  3.818e-01, ...,  3.822e-01,  3.822e-01]))